<a href="https://colab.research.google.com/github/2653319/book-example/blob/main/%E7%8C%9C%E6%8B%B3%E8%BE%A8%E8%AD%98_%E5%A4%9A%E5%85%83%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C2/W4/ungraded_lab/C2_W4_Lab_1_multi_class_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

未分級實驗室：多級分類器
在本實驗中，您將了解如何構建模型來區分兩個以上的類。該代碼將與您之前使用的代碼相似，只是對模型和訓練參數進行了一些關鍵更改。讓我們潛入吧！

##需更改的函式

此猜拳為多元，以往都是二元

1.模型的輸出，激活函式需更改，由sigmoid(二元) -> softmax(多元)

2.圖片生產器得class_mod需更改，由binary(二元) -> categorical(多元)

3.模型的loss需更改，由binary_crossentropy(二元) -> categorical_crossentropy(多元) 



## 下載並準備數據集

您將使用 [Rock-Paper-Scissors 數據集](http://www.laurencemoroney.com/rock-paper-scissors-dataset/)，這是一個包含石頭、紙和剪刀姿勢的手圖像庫。

In [ ]:
# Download the train set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps.zip
    
# Download the test set
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps-test-set.zip

In [ ]:
import zipfile

# Extract the archive
local_zip = './rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-train')
zip_ref.close()

local_zip = './rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('tmp/rps-test')
zip_ref.close()

像往常一樣，您會將目錄名稱分配給變量，並將文件名作為健全性檢查。

In [ ]:
import os

base_dir = 'tmp/rps-train/rps'

rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

print('total training rock images:', len(os.listdir(rock_dir)))
print('total training paper images:', len(os.listdir(paper_dir)))
print('total training scissors images:', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

您還可以檢查一些圖像以查看模型輸入的多樣性。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

##建立模型

然後，您將構建您的 CNN。您將使用 4 個卷積層和 64-64-128-128 過濾器，然後附加一個“Dropout”層以避免過度擬合，並添加一些 Dense 層用於分類。輸出層將是由 [Softmax] (https://www.tensorflow.org/api_docs/python/tf/nn/softmax) 激活的 3 神經元密集層。當您在 Fashion MNIST 進行培訓時，您已經在課程 1 中看到了這一點。它將你的輸出縮放為一組加起來為 1 的概率。這個 3 神經元輸出的順序將是 `paper`-`rock`-`scissors`（例如，`[0.8 0.2 0.0]` 輸出表示模型預測紙張的概率為 80%，岩石的概率為 20%。

You can examine the architecture with `model.summary()` below.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  
])

# Print the model summary
model.summary()

然後，您將編譯模型。這裡的關鍵變化是 `loss` 函數。而在您將 `binary_crossentropy` 用於 2 個類之前，您將其更改為 [categorical_crossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-function) 以將其擴展到更多類。

In [ ]:
# Set the training parameters
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

準備 ImageDataGenerator
您將像以前一樣準備生成器。您將為數據增強設置訓練集，以便它可以模仿模型需要學習的其他姿勢。

In [ ]:
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "tmp/rps-train/rps"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "tmp/rps-test/rps-test-set"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

訓練模型並評估結果
您將訓練 25 個 epoch，然後評估結果。觀察訓練和驗證的準確性如何呈上升趨勢。這很好地表明該模型僅對您的訓練集沒有過度擬合。


In [ ]:
# Train the model
history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

#  模型預測

您應該能夠在此處上傳圖像並對其進行分類而不會崩潰。但是，此代碼塊僅適用於 Google Colab。您可以使用自己的圖像或使用可用的圖像 [此處](https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-validation.zip)

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
## CODE BLOCK FOR NON-SAFARI BROWSERS
## SAFARI USERS: PLEASE SKIP THIS BLOCK AND RUN THE NEXT ONE INSTEAD

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(fn)
  print(classes)

`Safari` users will need to upload the images(s) manually in their workspace. Please follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# # CODE BLOCK FOR SAFARI USERS

# import os
# import numpy as np
# from keras.preprocessing import image

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#     print()
#     # predicting images
#     path = '/tmp/images/' + i
#     img = image.load_img(path, target_size=(150, 150))
#     x = image.img_to_array(img)
#     x = np.expand_dims(x, axis=0)
    
#     images = np.vstack([x])
#     classes = model.predict(images, batch_size=10)
#     print(path)
#     print(classes)

##包起來

關於多類分類器的這個簡短練習到此結束。您已經看到，只需進行一些更改，您就能夠轉換您的二元分類器來預測更多類。您在數據和模型準備中使用了相同的技術，並且能夠在 25 個 epoch 中獲得相對較好的結果。為了練習，您可以搜索其他數據集（例如 [here](https://archive.ics.uci.edu/ml/datasets.php) 具有更多類並修改模型以適應它。嘗試嘗試不同的層和數據增強技術來改善您的指標。